In [1]:
! pip install treefarms

Defaulting to user installation because normal site-packages is not writeable
distutils: /hpc/home/ts518/.local/lib/python3.9/site-packages
sysconfig: /hpc/home/ts518/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [2]:
import treefarms # check if this works

In [3]:
import pandas as pd
import numpy as np
import pathlib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from treefarms.model.threshold_guess import compute_thresholds, cut
from treefarms import TREEFARMS
from treefarms.model.model_set import ModelSetContainer

# Example

In this example, we run TREEFARMS on COMPAS, a recidivism dataset. The COMPAS dataset contains 6907 samples and 7 continuous features. We visualize the Rashomon set using `timbertrek` package, as well as show the way to obtain individual trees from the Rashomon set.


In [4]:
import os
os.getcwd()

'/hpc/group/csdept/ts518'

In [5]:
# read the dataset
df = pd.read_csv("/hpc/group/csdept/ts518/treeFarms/experiments/datasets/compas/binned.csv")
X, y = df.iloc[:, :-1], df.iloc[:, -1]
h = df.columns[:-1]
df


,sex:Female,age:<21,age:<23,age:<26,age:<46,juvenile-felonies:=0,juvenile-misdemeanors:=0,juvenile-crimes:=0,priors:=0,priors:=1,priors:2-3,priors:>3,recidivate-within-two-years:1
0,0,0,0,0,0,1,1,1,1,0,0,0,0
1,0,0,0,0,1,1,1,1,1,0,0,0,1
2,0,0,1,1,1,1,1,0,0,0,0,1,1
3,0,0,0,0,1,1,1,1,1,0,0,0,0
4,0,0,0,0,1,1,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6902,0,0,1,1,1,1,1,1,1,0,0,0,0
6903,0,0,1,1,1,1,1,1,1,0,0,0,0
6904,0,0,0,0,0,1,1,1,1,0,0,0,0
6905,1,0,0,0,1,1,1,1,0,0,1,0,0


We fit the Rashomon set on the COMPAS dataset.


In [6]:
# train TREEFARMS model
config = {
    "regularization": 0.01,  # regularization penalizes the tree with more leaves. We recommend to set it to relative high value to find a sparse tree.
    "rashomon_bound_multiplier": 0.05,  # rashomon bound multiplier indicates how large of a Rashomon set would you like to get
}

model = TREEFARMS(config)

model.fit(X, y)


null
Finding Optimal Objective...
treefarms reported successful execution
training completed. Number of trees in the Rashomon set: 1422
{
  "false": {
    "false": {
      "complexity": 0.009999999776482582,
      "loss": 0.021861879155039787,
      "name": "recidivate-within-two-years:1",
      "prediction": 1
    },
    "feature": 7,
    "name": "juvenile-crimes:=0",
    "reference": 1,
    "relation": "==",
    "true": {
      "complexity": 0.009999999776482582,
      "loss": 0.21644708514213562,
      "name": "recidivate-within-two-years:1",
      "prediction": 0
    },
    "type": "integral"
  },
  "feature": 11,
  "model_objective": 0.3748675286769867,
  "name": "priors:>3",
  "reference": 1,
  "relation": "==",
  "true": {
    "complexity": 0.009999999776482582,
    "loss": 0.10655856132507324,
    "name": "recidivate-within-two-years:1",
    "prediction": 1
  },
  "type": "integral"
}
{
  "false": {
    "complexity": 0.009999999776482582,
    "loss": 0.03938033804297447,
    "n

We then visualize the Rashomon set. 

In [7]:
# TREEFARMS will attempt to obtain feature names from the DataFrame columns.
# However, it is also possible to manually set this value, such as the
# commented code snippet below

feature_names = df.columns

feature_description = {
    "sex": {"info": "Sex", "type": "is", "short": "Sex"},
    "age": {"info": "Age", "type": "count", "short": "Age"},
    "juvenile-felonies": {
        "info": "Number of juvenile felonies",
        "type": "count",
        "short": "Juv felony",
    },
    "juvenile-misdemeanors": {
        "info": "Number of juvenile misdemeanors",
        "type": "count",
        "short": "Juv misdemeanor",
    },
    "juvenile-crimes": {
        "info": "Number of juvenile crimes",
        "type": "count",
        "short": "Juv crime",
    },
    "priors": {
        "info": "Number of prior crimes",
        "type": "count",
        "short": "Prior crime",
    },
    "recidivate-within-two-years": {
        "info": "Has recidivated within two years",
        "type": "yes",
        "short": "Recidivated",
    },
}
model.visualize(feature_names, feature_description)

# model.visualize() # default visualization


Generating trie from 1422 trees:   0%|          | 0/1422 [00:00<?, ?it/s]

Generating decision paths from 1422 trees: 100%|██████████| 1422/1422 [00:07<00:00, 187.35it/s]


<iframe
 srcdoc="<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0" /><title>TimberTrek</title><style>html{font-size:16px;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;text-rendering:optimizeLegibility;-webkit-text-size-adjust:100%;-moz-text-size-adjust:100%}html,body{position:relative;width:100%;height:100%}body{margin:0;padding:0;box-sizing:border-box;font-family:-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Oxygen-Sans,Ubuntu,Cantarell,Helvetica Neue,sans-serif;color:#4a4a4a;font-size:1em;font-weight:400;line-height:1.5}*,:after,:before{box-sizing:inherit}a{color:#0064c8;text-decoration:none}a:hover{text-decoration:underline}a:visited{color:#0050a0}label{display:block}input,button,select,textarea{font-family:inherit;font-size:inherit;-webkit-padding:.4em 0;padding:.4em;margin:0 0 .5em;box-sizing:border-box;border:1px solid #ccc;border-radius:2px}input:disabled{color:#ccc}button{color:#333;background-color:#f4f4f4;outline:none}button:disabled{color:#999}button:not(:disabled):active{background-color:#ddd}button:focus{border-color:#666}</style><script defer src='...'></script><script defer src='...'></script></head><body></body></html>"
 frameBorder="0"
 width="100%"
 height="650px"
 id="timbertrek-iframe-3977107">

It is also possible to obtain individual trees from the Rashomon set. The following cell demonstrates getting the accuracy of the first tree in the Rashomon set as well as printing out its structure.

In [8]:
first_tree = model[0]
print(f'The accuracy of the first tree on the data is: {first_tree.score(X, y)}')
print(model[0])

The accuracy of the first tree on the data is: 0.6565802808744752
if feature_11 = true then:
    predicted Prediction: 1

else if feature_11 != true and feature_7 = true then:
    predicted Prediction: 0

else if feature_0 = true and feature_11 != true and feature_7 != true and feature_8 = true then:
    predicted Prediction: 1

else if feature_0 = true and feature_11 != true and feature_7 != true and feature_8 != true then:
    predicted Prediction: 0

else if feature_0 != true and feature_11 != true and feature_7 != true then:
    predicted Prediction: 1


Thank you for reading our tutorial. Please do try out our methods with different parameters and datasets. Happy tree farming!


## my methods

### mostly using this part of the notebook for testing and stuff

In [9]:
import json

In [10]:
feature_names

Index(['sex:Female', 'age:<21', 'age:<23', 'age:<26', 'age:<46',
       'juvenile-felonies:=0', 'juvenile-misdemeanors:=0',
       'juvenile-crimes:=0', 'priors:=0', 'priors:=1', 'priors:2-3',
       'priors:>3', 'recidivate-within-two-years:1'],
      dtype='object')

In [11]:
tree_dict = json.loads(model[0].json())
tree_dict

{'feature': 11,
 'relation': '==',
 'reference': 'true',
 'true': {'prediction': 1, 'name': 'Prediction'},
 'false': {'feature': 7,
  'relation': '==',
  'reference': 'true',
  'true': {'prediction': 0, 'name': 'Prediction'},
  'false': {'feature': 0,
   'relation': '==',
   'reference': 'true',
   'true': {'feature': 8,
    'relation': '==',
    'reference': 'true',
    'true': {'prediction': 1, 'name': 'Prediction'},
    'false': {'prediction': 0, 'name': 'Prediction'}},
   'false': {'prediction': 1, 'name': 'Prediction'}}}}

In [12]:
def go_to_depth(tree_json, depth):
    # tree is json version of tree classifier object

    if type(tree_json) != dict:
        tree_json = json.loads(tree_json.json())

    # base case, tree is just a leaf
    if "true" not in tree_json:
        return 
    # correct depth reached
    if depth == 1:
        return [tree_json]
    if depth == 2:
        return [tree_json["true"], tree_json["false"]]
    # recursion, keep going
    else: 
        left_tree = go_to_depth(tree_json["true"], depth-1)
        right_tree = go_to_depth(tree_json["false"], depth-1)
        if not left_tree:
            if right_tree:
                return right_tree
            return
        if not right_tree:
            if left_tree:
                return left_tree
            return
        else:
            return left_tree + right_tree

In [13]:
# testing depth function on first tree
go_to_depth(tree_dict, depth=4)

[{'feature': 8,
  'relation': '==',
  'reference': 'true',
  'true': {'prediction': 1, 'name': 'Prediction'},
  'false': {'prediction': 0, 'name': 'Prediction'}},
 {'prediction': 1, 'name': 'Prediction'}]

In [14]:
def entropy(ps):
    """
    Calculate the entropy of a given list of binary labels.
    ps[0] looks like it's the proportion of positive labels, 
    ps[1] is proportion of negative ones.
    """
    p_positive = ps[0]
    if p_positive == 0 or p_positive == 1:
        return 0  # Entropy is 0 if all labels are the same
    entropy_val = - (p_positive * np.log2(p_positive) +
                        (1 - p_positive) * np.log2(1 - p_positive))
    return entropy_val

In [15]:
def find_best_feature_to_split_on(X_train,y_train):
        num_features = X_train.shape[1]
        max_gain = -10
        gain_of_feature_to_split = 0
        p_original = np.mean(y_train) # mean of all training labels (aka percentage that are pos)
        entropy_original = entropy([p_original, 1-p_original]) # get entropy of all data (basically treating it like a tree with 1 leaf)
        best_feature = -1
        for feature in range(num_features):
            # Left child labels
            p_left = np.mean(y_train[X_train.iloc[:, feature] == 1])
            
            # Right child labels
            p_right = np.mean(y_train[X_train.iloc[:, feature] == 0])

            p_left = 0 if np.isnan(p_left) else p_left
            p_right = 0 if np.isnan(p_right) else p_right
        
            entropy_left = entropy(np.array([p_left, 1 - p_left]))
            
            entropy_right = entropy(np.array([p_right, 1 - p_right]))
            
            proportion_of_examples_in_left_leaf = (np.sum(X_train.iloc[:, feature] == 1) / len(X_train))
            proportion_of_examples_in_right_leaf = (np.sum(X_train.iloc[:, feature] == 0) / len(X_train))
            gain = entropy_original - ( proportion_of_examples_in_left_leaf* entropy_left +
                                        proportion_of_examples_in_right_leaf* entropy_right)
            if gain >= max_gain:
                max_gain = gain
                best_feature = feature

        return best_feature

In [16]:
find_best_feature_to_split_on(X, y) # this works

11

In [17]:
def is_greedy(tree, X_train, y_train, verbose = 0):
    # check if first split of tree is greedy or not.
    # if tree is empty?

    # if tree is a leaf, all leaves are considered greedy
    if "prediction" in tree:
        return True
    # else check if it split on the greediest feature
    else:
        best_feature = find_best_feature_to_split_on(X_train, y_train)
        if verbose == 1:
            print("Feature for greediest split:", best_feature)
            print("Feature tree makes split on:", tree["feature"])
        if tree["feature"] == best_feature:
            return True
        return False

In [20]:
is_greedy(tree_dict, X, y) # works: best feature is #11, the first split is on 11

True

In [21]:
def check_greedy(tree, X_train, y_train, depth, verbose=0):
    # tree is tree classifier object -> json
    # X_train, y_train are data and labels
    # depth is how deep to start checking
    # Returns True (tree is greedy past depth) or False

    # step 0. turn tree classifier into json
    if type(tree) != dict:
        tree_json = json.loads(tree.json())
    else:
        tree_json = tree
    if verbose:
        print("Current depth:", depth)
    # step 1. go to specified depth of tree, get left and right trees
    subtrees = go_to_depth(tree_json, depth)
    if not subtrees: # went past the max depth of tree
        return False
    elif len(subtrees) > 1:  # left and right subtrees exist
        left_tree, right_tree = subtrees[0], subtrees[1]
        if not is_greedy(left_tree, X_train, y_train, verbose):
            return False
        if not is_greedy(right_tree, X_train, y_train, verbose):
            return False

        if "prediction" in left_tree: # check if left tree is leaf
            left_greedy = True
        else:
            left_split_feature = left_tree["feature"]
            X_train_left = X_train[X_train.iloc[:, left_split_feature] == True]
            y_train_left = y_train[X_train.iloc[:, left_split_feature] == True]
            left_greedy = check_greedy(left_tree, X_train_left, y_train_left, depth=depth + 1, verbose=verbose)

        if "prediction" in right_tree: # check if right tree is leaf
            right_greedy = True
        else:
            right_split_feature = right_tree["feature"]
            X_train_right = X_train[X_train.iloc[:, right_split_feature] == False]
            y_train_right = y_train[X_train.iloc[:, right_split_feature] == False]
            right_greedy = check_greedy(right_tree, X_train_right, y_train_right, depth=depth + 1, verbose=verbose)

        return (left_greedy and right_greedy)

    elif len(subtrees) == 1:  # only 1 subtree
        left_tree = subtrees[0]

        if not is_greedy(left_tree, X_train, y_train, verbose):
            return False

        if "prediction" in left_tree:
            return True
        else:
            left_split_feature = left_tree["feature"]
            X_train_left = X_train[X_train.iloc[:, left_split_feature] == True]
            y_train_left = y_train[X_train.iloc[:, left_split_feature] == True]
            return check_greedy(left_tree, X_train_left, y_train_left, 
                                depth=depth + 1, verbose=verbose)

In [23]:
check_greedy(first_tree, X, y, depth=2) # runs successfully for depth 2
# could just try training some trees i know are greedy?
# it works for depth 5 too (leaves only, returns true)

False

In [24]:
test_check_greedy = go_to_depth(tree_dict, depth=2)
test_check_greedy # left is a leaf, splits on feature 7 on the right

[{'prediction': 1, 'name': 'Prediction'},
 {'feature': 7,
  'relation': '==',
  'reference': 'true',
  'true': {'prediction': 0, 'name': 'Prediction'},
  'false': {'feature': 0,
   'relation': '==',
   'reference': 'true',
   'true': {'feature': 8,
    'relation': '==',
    'reference': 'true',
    'true': {'prediction': 1, 'name': 'Prediction'},
    'false': {'prediction': 0, 'name': 'Prediction'}},
   'false': {'prediction': 1, 'name': 'Prediction'}}}]

In [25]:
check_greedy(model[0], X, y, depth=1, verbose=1)

Current depth: 1
Feature for greediest split: 11
Feature tree makes split on: 11
Current depth: 2
Feature for greediest split: 7
Feature tree makes split on: 7
Current depth: 3
Feature for greediest split: 2
Feature tree makes split on: 8


False

In [26]:
greedy_list = []
for i in range(model.model_set.get_tree_count()):
    greedy_list.append(check_greedy(model[i], X, y, depth=1))

In [28]:
tree_ind = np.nonzero(greedy_list)[0].tolist()

In [29]:
print("Number of greedy trees at depth 1:", sum(greedy_list))
print("Tree numbers:", tree_ind)

Number of greedy trees at depth 1: 8
Tree numbers: [9, 11, 103, 319, 985, 1036, 1084, 1096]


In [30]:
for i in tree_ind:
    print(check_greedy(model[i], X, y, depth=1, verbose=1))
    print()

Current depth: 1
Feature for greediest split: 11
Feature tree makes split on: 11
Current depth: 2
Feature for greediest split: 7
Feature tree makes split on: 7
Current depth: 3
True

Current depth: 1
Feature for greediest split: 11
Feature tree makes split on: 11
Current depth: 2
Feature for greediest split: 7
Feature tree makes split on: 7
Current depth: 3
True

Current depth: 1
Feature for greediest split: 11
Feature tree makes split on: 11
Current depth: 2
Feature for greediest split: 7
Feature tree makes split on: 7
Current depth: 3
True

Current depth: 1
Feature for greediest split: 11
Feature tree makes split on: 11
Current depth: 2
Feature for greediest split: 7
Feature tree makes split on: 7
Current depth: 3
True

Current depth: 1
Feature for greediest split: 11
Feature tree makes split on: 11
Current depth: 2
Feature for greediest split: 7
Feature tree makes split on: 7
Current depth: 3
True

Current depth: 1
Feature for greediest split: 11
Feature tree makes split on: 11
Curr